### BioGRID interaction retrieval

In [2]:
import requests, sys, json
import pandas as pd
import pickle

In [3]:
### BioGRID access key
BIOGRID_ACCESS_KEY = 'a207ffa23db79e55a7d7a5a69b79f0c9'

### BioGRID server
biogrid_server = f'https://webservice.thebiogrid.org/interactions/?accesskey={BIOGRID_ACCESS_KEY}'

In [4]:
with open('../Results/GENIE3/SARS-CoV-2/interactions_dataframe_with_names.pickle', 'rb') as f:
    interactions = pickle.load(f)

interactions

,gene1,gene2,score,gene1_name,gene2_name
0,ENSG00000200336,ENSG00000201695,0.102868,5S_rRNA,5S_rRNA
1,ENSG00000201695,ENSG00000200336,0.085720,5S_rRNA,5S_rRNA
2,ENSG00000273590,ENSG00000205670,0.081955,LOC102723553,LOC102723553
3,ENSG00000205670,ENSG00000273590,0.077385,LOC102723553,LOC102723553
4,ENSG00000160201,ENSG00000275895,0.072684,LOC102724594,LOC102724594
...,...,...,...,...,...
249495,ENSG00000137673,ENSG00000201965,0.000000,MMP7,Y_RNA
249496,ENSG00000206918,ENSG00000199546,0.000000,U6,Y_RNA
249497,ENSG00000252714,ENSG00000240665,0.000000,5S_rRNA,LSP1P2
249498,ENSG00000228078,ENSG00000202354,0.000000,HLA-U,RNY3


In [5]:
top_ints = interactions[['gene1_name', 'gene2_name']].iloc[:100].values
top_ints

array([['5S_rRNA', '5S_rRNA'],
       ['5S_rRNA', '5S_rRNA'],
       ['LOC102723553', 'LOC102723553'],
       ['LOC102723553', 'LOC102723553'],
       ['LOC102724594', 'LOC102724594'],
       ['PKD1P2', 'ENSG00000285628'],
       ['LOC124908527', 'LOC124908527'],
       ['LOC124908527', 'LOC124908527'],
       ['ENSG00000224114', 'RPS14P8'],
       ['HLA-DQB2', 'RPL13P12'],
       ['LOC124908527', 'LOC124908527'],
       ['LINC01806', 'CD200R1L-AS1'],
       ['LOC124908527', 'LOC124908527'],
       ['LOC124908527', 'LOC124908527'],
       ['THNSL2', 'F8A3'],
       ['LOC124908527', 'LOC124908527'],
       ['LOC124908527', 'LOC124908527'],
       ['LOC124908527', 'LOC124908527'],
       ['LOC124908527', 'LOC124908527'],
       ['LDHAP4', 'TEKT4P2'],
       ['LOC124908527', 'LOC124908527'],
       ['LOC124908527', 'LOC124908527'],
       ['XIST', 'TTTY15'],
       ['LOC124908527', 'LOC124908527'],
       ['ENSG00000285628', 'PKD1P2'],
       ['LOC105376805', 'ENSG00000278144'],
       ['

In [16]:
gene1, gene2 = top_ints[0]
print(gene1, gene2)
# for gene1, gene2 in top_ints:
# the rules outlined in the Wiki: https://wiki.thebiogrid.org/doku.php/biogridrest
params = {
    "accesskey": BIOGRID_ACCESS_KEY,
    "format": "json",  # Return results in TAB2 format
    "geneList": "|".join(gene1),  # Must be | separated
    "searchNames": "true",  # Search against official names
    "includeInteractors": "false",  # Set to true to get any interaction involving EITHER gene, set to false to get interactions between genes
    "includeHeader": "true",
}

# Additional options to try, you can uncomment them as necessary
# params["start"] = 5 # Specify where to start fetching results from if > 10,000 results being returned
# params["max"] = 10 # Specify the number of results to return, max is 10,000
# params["interSpeciesExcluded"] = "false" # true or false, If ‘true’, interactions with interactors from different species will be excluded (ex. no Human -> Mouse interactions)
# params["selfInteractionsExcluded"] = "false" # true or false, If ‘true’, interactions with one interactor will be excluded. (ex. no STE11 -> STE11 interactions)
# params["searchIds"] = "false" # true or false, If ‘true’, ENTREZ_GENE, ORDERED LOCUS and SYSTEMATIC_NAME (orf) will be examined for a match with the geneList
# params["searchSynonyms"] = "false" # true or false, If ‘true’, SYNONYMS will be examined for a match with the geneList
# params["searchBiogridIds"] = "false" # true or false, If ‘true’, BIOGRID INTERNAL IDS will be examined for a match with the geneList
# params["excludeGenes"] = "false" # true or false, If 'true' the geneList becomes a list of genes to EXCLUDE rather than to INCLUDE
# params["includeInteractorInteractions"] = "true" # true or false, If ‘true’ interactions between the geneList’s first order interactors will be included. Ignored if includeInteractors is ‘false’ or if excludeGenes is set to ‘true’.
# params["htpThreshold"] = 50 # Any publication with more than this many interactions will be excluded
# params["throughputTag"] = "any" # any, low, high. If set to low, only `low throughput` interactions will be returned, if set to high, only `high throughput` interactions will be returned
# params["additionalIdentifierTypes"] = "SGD|FLYBASE|REFSEQ" # You can specify a | separated list of additional identifier types to search against (see get_identifier_types.py)

r = requests.get(biogrid_server, params=params)
interactions = r.json()

# Create a hash of results by interaction identifier
data = {}
for interaction_id, interaction in interactions.items():
    data[interaction_id] = interaction
    # Add the interaction ID to the interaction record, so we can reference it easier
    data[interaction_id]["INTERACTION_ID"] = interaction_id

# Load the data into a pandas dataframe
dataset = pd.DataFrame.from_dict(data, orient="index")

columns = [
    "INTERACTION_ID",
    "SYSTEMATIC_NAME_A",
    "SYSTEMATIC_NAME_B",
    "OFFICIAL_SYMBOL_A",
    "OFFICIAL_SYMBOL_B",
    "EXPERIMENTAL_SYSTEM",
    "PUBMED_ID",
    "PUBMED_AUTHOR",
    "THROUGHPUT",
    "QUALIFICATIONS",
]
# dataset = dataset[columns]
dataset


5S_rRNA 5S_rRNA


,BIOGRID_INTERACTION_ID,ENTREZ_GENE_A,ENTREZ_GENE_B,BIOGRID_ID_A,BIOGRID_ID_B,SYSTEMATIC_NAME_A,SYSTEMATIC_NAME_B,OFFICIAL_SYMBOL_A,OFFICIAL_SYMBOL_B,SYNONYMS_A,...,ORGANISM_A,ORGANISM_B,THROUGHPUT,QUANTITATION,MODIFICATION,ONTOLOGY_TERMS,QUALIFICATIONS,TAGS,SOURCEDB,INTERACTION_ID
2298858,2298858,31293,33281,57823,59534,Dmel_CG3936,Dmel_CG4385,N,S,1.1|16-178|16-55|Ax|CG3936|CT13012|Chp|Co|Dmel...,...,7227,7227,Low Throughput,-,-,{},-,-,FLYBASE,2298858
2312157,2312157,31293,31293,57823,57823,Dmel_CG3936,Dmel_CG3936,N,N,1.1|16-178|16-55|Ax|CG3936|CT13012|Chp|Co|Dmel...,...,7227,7227,Low Throughput,-,-,{},-,-,FLYBASE,2312157
2476974,2476974,31293,31293,57823,57823,Dmel_CG3936,Dmel_CG3936,N,N,1.1|16-178|16-55|Ax|CG3936|CT13012|Chp|Co|Dmel...,...,7227,7227,Low Throughput,-,-,{},-,-,FLYBASE,2476974
2652709,2652709,43740568,43740568,4383848,4383848,GU280_gp02,GU280_gp02,S,S,spike|SARS-CoV2 S|SARS-CoV2 spike|S protein|su...,...,2697049,2697049,Low Throughput,-,-,{},EM structure of the SARS-CoV-2 spike (S) glyc...,-,BIOGRID,2652709
2652712,2652712,43740568,43740568,4383848,4383848,GU280_gp02,GU280_gp02,S,S,spike|SARS-CoV2 S|SARS-CoV2 spike|S protein|su...,...,2697049,2697049,Low Throughput,-,-,{},SARS-CoV-2 spike (S) glycoprotein trimer,-,BIOGRID,2652712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3739735,3739735,43740575,43740575,4383847,4383847,GU280_gp10,GU280_gp10,N,N,nuc|nucleocapsid|SARS-CoV2 N|N protein|ncap|SA...,...,2697049,2697049,Low Throughput,-,-,{},-,-,BIOGRID,3739735
3739737,3739737,43740568,43740568,4383848,4383848,GU280_gp02,GU280_gp02,S,S,spike|SARS-CoV2 S|SARS-CoV2 spike|S protein|su...,...,2697049,2697049,Low Throughput,-,-,{},cryoEM structures of trimeric SARS-CoV-2 Omicr...,-,BIOGRID,3739737
3739763,3739763,43740568,43740568,4383848,4383848,GU280_gp02,GU280_gp02,S,S,spike|SARS-CoV2 S|SARS-CoV2 spike|S protein|su...,...,2697049,2697049,Low Throughput,-,-,{},cryoEM structures of trimeric SARS-CoV-2 WT 6P...,-,BIOGRID,3739763
3739772,3739772,43740568,43740568,4383848,4383848,GU280_gp02,GU280_gp02,S,S,spike|SARS-CoV2 S|SARS-CoV2 spike|S protein|su...,...,2697049,2697049,Low Throughput,-,-,{},cryoEM structures of trimeric SARS-CoV-2 Omicr...,-,BIOGRID,3739772
